<a href="https://colab.research.google.com/github/asdlkjw/SK_shieldus/blob/main/watering_%EC%9E%90%EB%8F%99%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import NaN
import zipfile
import os
import datetime
import pandas as pd
from tqdm.notebook import tqdm as td
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import sqlite3
import warnings
warnings.filterwarnings('ignore')


today = datetime.date.today()

m = today.month
d = today.day

def zip_ext(day):    # zip파일 해제 및 DataFrame 변환
  if (f"{day}└╧") in os.listdir(f"./") :
    print('already unzip')
    df_all = pd.DataFrame()
    return df_all
  else:
    fantasy_zip = zipfile.ZipFile(f'/content/drive/MyDrive/{day}일.zip')
    fantasy_zip.extractall('./')
    fantasy_zip.close()
  zipdir = f"{day}└╧"
  file_num = os.listdir(f"./{zipdir}")
  dirlist = file_num
  print(dirlist)

  df_all = pd.DataFrame()
  for i,j in td(enumerate(dirlist)):
    if j != '.ipynb_checkpoints':
      df = pd.read_csv(f"{zipdir}/{j}", encoding= 'euc-kr')
      df_all = pd.concat([df_all, df])
      
  return df_all

def top_all(data, input_day:int):  # url 접속 건수 카운터
  top =  Counter(data)
  top = (pd.DataFrame.from_dict(dict(top), orient='index').rename(columns={0:'count'}))
  top = top.reset_index()
  top.columns = ['url','count']
  top = top.sort_values(by = 'count', ascending= False)
  top.reset_index(drop = True, inplace= True)
  top.to_csv(f'./top_all_{input_day}.csv')
  print(f'top_all_{input_day}.csv -> save complete')
  top = top[:1000]
  top.reset_index(drop=True, inplace= True)
  return top

def db_connect(table = None, mal_data:list = [] ,db_path = "/content/drive/MyDrive/watering.db"): # dbms

  # 데이터 베이스 연결
  db = sqlite3.connect(db_path)
  print(f"====db connect {table}====")

  if table == 'shodan':
    #db 테이블 없으면 생성
    db.execute(f"""
            CREATE TABLE IF NOT EXISTS new_shodan_list (
                tmp INTEGER , 
                src TEXT,
                name TEXT,
                cve TEXT,
                PRIMARY KEY("tmp" AUTOINCREMENT)
            )""")

    db.commit()

    #db 저장
    if mal_data != []:
      mal_url, mal_explain, mal_cve = mal_data
      # mal_cve = mal_cve.split(' ')
      db.execute(f"""
        INSERT INTO new_shodan_list (src,name, cve) VALUES ("{mal_url}","{mal_explain}", "{mal_cve}")""")
      db.commit()
      print(f'{mal_explain} CVE save')

    #db shodan table 리스트 확인
    prev_data = db.execute(f"SELECT * FROM new_shodan_list").fetchall()
    db.commit()

    prev_data = pd.DataFrame(prev_data, columns=["tmp","src","name","cve"])

    return prev_data

  elif table == 'drop':
    #db 테이블 없으면 생성
    db.execute(f"""
            CREATE TABLE IF NOT EXISTS drop_list (
                tmp INTEGER , 
                src TEXT,
                PRIMARY KEY("tmp" AUTOINCREMENT)
            )""")

    db.commit()

    #db 저장
    if mal_data != []:
      mal_url = mal_data
      for i in mal_url:
        db.execute(f"""
          INSERT INTO drop_list (src) VALUES ("{i}")""")
        db.commit()

    #db drop table 리스트 확인
    drop_data = db.execute(f"SELECT * FROM drop_list").fetchall()
    db.commit()
    drop_data = pd.DataFrame(drop_data, columns=["tmp","src"])
  
    return drop_data

  return print('please input data (table, mal_data)')

def mal_check(top, prev_data):
  mal_src = list()
  mal = list()
  for i in top.url:
    try:
      mal_src.append(prev_data[prev_data.src.str.match(f'{i}'.strip())].name.values[0])
      if mal_src[-1] != '':
        mal.append(1)
      else:
        mal.append(0)
    except:
      mal_src.append(' ')
      mal.append(0)
  top['mal_src'] = mal_src
  top['mal'] = mal
  
  return top

def dst_ip_check(df, top):
  dst_ip_list = list()
  for i in td(top[:20].url):
    if  i in top[:20][top[:20].mal == 1].url:
      continue
    dst_ip = df[(df.url.str.contains('\/\/'+f'{i}'.strip(), na = False)) & (df['HTTP 메소드'].isin(["GET","POST"])) & (df['http_status_code'].isin(['200','204']) )]['목적지IP'].unique()
    dst_ip_list.append(dst_ip[:2])
  find_shodan = top[:20]
  find_shodan['dst_ip'] = dst_ip_list
  return find_shodan

def drop_not_mal(top, drop_data):
  for i in drop_data.src:
    drop_list = top[top.url.str.contains(f'{i.strip()}')].index  
    top.drop(index = drop_list, inplace= True)

  # for i in shodan_drop:
  #   aa = top100[top100.url.str.contains(f'{i.strip()}')].index  
  #   top100.drop(index = aa, inplace= True)
  return top
#---------------------------------------------------------------------------------------------------------------------------


temp= False
while True:
  try:
    input_day = int(input(f'압축파일명 날짜(Day)를 입력 : '))
    df_all = zip_ext(input_day)
    if df_all.shape[0] == 0:
      top = pd.read_csv(f'./top_all_{input_day}.csv', encoding= 'euc-kr')
      df = pd.read_csv("./df_temporary.csv")
      break
  except:
    print('파일이 존재하지 않습니다.')
  else:
    print('c')
    df = df_all[['장비발생시간','HTTP 메소드','http_status_code','출발지IP','목적지IP','Action','url']]
    df[:5000].to_csv('./df_temporary.csv', index= False)
    tmp = []
    for i in df.url:
      try:
        tmp.append(i.split('//')[1])
      except:
        tmp.append(i)
    tmp2 = []
    for i in tmp:
      try:
        tmp2.append(i.split('/')[0])
      except:
        tmp2.append(i)

    df['url2'] = tmp2
    print(f'url 합계:{len(df.url2.unique())}')


    top = top_all(df.url2, input_day)
    break
#---------------------------------------------------------------------------------------------------------------------------

drop_data = db_connect(table = 'drop')
top = drop_not_mal(top, drop_data)

top = top.sort_values(ascending=False, by= 'count').reset_index(drop = True)

prev_data = db_connect(table = 'shodan')
top = mal_check(top, prev_data)

find_shodan = dst_ip_check(df, top)

#---------------------------------------------------------------------------------------------------------------------------

exit = False
while exit != True:
  print("========watering_top_list========")
  print(find_shodan)
  print()

  while True:
    malware = input('you found malware? (Y or N or exit): ')
    if malware in ['Y','y','N','n','exit','exit()']:
      break
    else:
      print('(Y or N or exit) 안에서 입력해주세요.')

  if malware in ['Y','y']:
    while True:
      mal_url     = input('please input url     : ').strip()
      mal_explain = input('please input explain : ').strip()
      mal_cve     = input('please input cve     : ').strip().split()
      check = input('다시입력 하시겠습니까? (y or n)')
      if check in ['y','Y']:
        pass
      else:
        break
    mal_data = [mal_url, mal_explain, mal_cve]

    prev_data = db_connect(table='shodan', mal_data = mal_data)
    top = mal_check(top, prev_data)
    find_shodan = dst_ip_check(df, top)

  elif malware in ['N', 'n']:
    not_mal = find_shodan[~(find_shodan.mal == 1)].url.values
    drop_data = db_connect(table = 'drop', mal_data= not_mal)
    top = drop_not_mal(top, drop_data)
    find_shodan = dst_ip_check(df, top)
    print(f'drop된 not_mal 수 : {len(not_mal)}')
  
  elif malware == 'exit':
    tmp = len(find_shodan[(find_shodan.mal == 1)])
    find_shodan[(find_shodan.mal == 1)].to_csv(f"./top_{tmp}_{input_day}.csv")
    print(f'top_{tmp}_{input_day}.csv -> save complete')
    print("========watering_end========")
    exit = True
  
